# Testing file 
### where we evaluate BEUTEL's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.beutel.models import *
from models.beutel.learning import train_loop as beutel_train

In [2]:
batch_size = 64
epochs = 1
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:

FAIR_COEFFS = [.2, .5, .7, 1.]

hidden_layer_specs = {'clas':[8], 'enc':[8], 'adv':[8]}

In [4]:
test_loop = 1

## Load data

In [5]:
data_name = 'adult'

In [6]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [7]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [8]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 112), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [9]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [10]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [11]:
header = "model_name", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### BEUTEL for DP

In [12]:
fairdef = 'DemPar'

for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)
    
        model = Beutel(xdim, ydim, adim, zdim, hidden_layer_specs, fairdef)

        ret = beutel_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['BEUTEL4DP', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.1448388695716858 | 0.5440177917480469 | 0.5475058555603027 | 0.9201741218566895 | 0.6901939655172413 | 0.6080487400530504 | -101.54302055702918
> 2 | 0.1423984169960022 | 0.5317417979240417 | 0.5227327942848206 | 0.9201741218566895 | 0.826467175066313 | 0.6720822281167109 | -101.54302055702918
> 3 | 0.14108078181743622 | 0.5289418697357178 | 0.5284976363182068 | 0.9201741218566895 | 0.8338859416445623 | 0.6584466180371353 | -101.54302055702918
> 4 | 0.13901732861995697 | 0.5225849747657776 | 0.527664840221405 | 0.9201741218566895 | 0.8364970159151194 | 0.6588610742705571 | -101.54302055702918
> 5 | 0.13674296438694 | 0.5156745910644531 | 0.527228593826294 | 0.9201741218566895 | 0.8378647214854111 | 0.6601044429708223 | -101.54302055702918
> 6 | 0.13605545461177826 | 0.5134332180023193 | 0.5263343453407288 | 0.9201741218566895 | 0.839481100795756 | 0.661223474801061 | -101.54302055702918
> 7 

### BEUTEL for Eq Opp

In [14]:
fairdef = 'EqOpp'

for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = Beutel(xdim, ydim, adim, zdim, hidden_layer_specs, fairdef)

        ret = beutel_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['BEUTEL4EqOpp', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0058588190004229546 | 0.5432302355766296 | 0.5288085341453552 | 0.9201741218566895 | 0.6884946949602122 | 0.5864555702917772 | -101.54302055702918
> 2 | -0.0014059869572520256 | 0.548180103302002 | 0.551641047000885 | 0.9201741218566895 | 0.8240218832891246 | 0.662425397877984 | -101.54302055702918
> 3 | -0.005373783875256777 | 0.5400211215019226 | 0.5532488822937012 | 0.9201741218566895 | 0.8269645225464191 | 0.667606100795756 | -101.54302055702918
> 4 | -0.009550941176712513 | 0.5298053026199341 | 0.5533152222633362 | 0.9201741218566895 | 0.8313577586206896 | 0.6671916445623343 | -101.54302055702918
> 5 | -0.012470726855099201 | 0.5226964950561523 | 0.5533937215805054 | 0.9201741218566895 | 0.8344661803713528 | 0.6675646551724138 | -101.54302055702918
> 6 | -0.014044043608009815 | 0.5188814401626587 | 0.553451418876648 | 0.9201741218566895 | 0.8355852122015915 | 0.666901525198939 | -101.54

## Saving into DF then CSV

In [15]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,0.2,0.850233,0.818917,0.942081,0.960377,0.834281,0.893804,0.901955,1703.0,55.0,82.0,119.0,2499.0,301.0,463.0,794.0
1,LAFTR4DP,0.5,0.850399,0.832913,0.965804,0.997981,0.841565,0.904435,0.918298,1697.0,61.0,77.0,124.0,2517.0,283.0,479.0,778.0
2,LAFTR4DP,0.7,0.848238,0.855747,0.954423,0.959451,0.851976,0.898203,0.900423,1690.0,68.0,75.0,126.0,2550.0,250.0,520.0,737.0
3,LAFTR4DP,1.0,0.846742,0.870077,0.951554,0.940750,0.858251,0.896094,0.891274,1676.0,82.0,73.0,128.0,2564.0,236.0,531.0,726.0
4,LAFTR4EqOdds,0.2,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
5,LAFTR4EqOdds,0.5,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
6,LAFTR4EqOdds,0.7,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
7,LAFTR4EqOdds,1.0,0.849069,0.847931,0.958040,0.973379,0.848500,0.900269,0.906984,1698.0,60.0,77.0,124.0,2544.0,256.0,515.0,742.0
8,LAFTR4EqOpp,0.2,0.845911,0.864881,0.949424,0.939942,0.855291,0.894683,0.890451,1667.0,91.0,69.0,132.0,2540.0,260.0,507.0,750.0
9,LAFTR4EqOpp,0.5,0.845911,0.864881,0.949424,0.939942,0.855291,0.894683,0.890451,1667.0,91.0,69.0,132.0,2540.0,260.0,507.0,750.0


In [16]:
result_df.to_csv(f'{data_name}-result/beutel-{epochs}.csv')